In [ ]:
# Load and preprocess data
data = load_stock_data("AAPL")
list_of_data = [bt.feeds.PandasData(dataname=data, datetime="Date", timeframe=bt.TimeFrame.Minutes)]

# Define prompt and system message
prompt = "Please create a strategy that buy when RSI is below 50 and sell when RSI is above 80."
system = f"""
You are a python developer that intends to make a workable strategy from human requirements.
Your task is to create a new BackTestStrategy that follows the below msg.
Note: Only return class strategy and change `execute` function only, nothing else

{prompt}
"""

# Generate and run the strategy
code_script = python_code_generate(system, list_of_data)
exec(code_script)

# Initialize cerebro and run the strategy
cerebro = run_cerebro(BackTestStrategy, list_of_data)
thestrat = cerebro[0]

# Print out the final result
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
print("Total point return:", cerebro.broker.getvalue() - cerebro.broker.startingcash)

figs = cerebro.plot(iplot=False, style="pincandle", width=600, height=400)

In [1]:
!pip install backtrader groq 
!pip install constant influxdb_client 
# !pip install database_utils
!pip install yfinance

     ---------------------------------------- 0.0/63.5 kB ? eta -:--:--
     ------------ ------------------------- 20.5/63.5 kB 682.7 kB/s eta 0:00:01
     ------------------------------ ------- 51.2/63.5 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 63.5/63.5 kB 682.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for constant: filename=constant-0.0.4-py3-none-any.whl size=82353 sha256=5e7849ddb1c5a40e2af595a5981e2d160f90050ada8b902d7d720ee7de824935
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\80\e3\27\938579a2acd36af224164c8618a465fca8a8d5118c1208b90a
Successfully built constant


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib

%matplotlib inline
from pprint import pprint
from groq import Groq
import os
# from database_utils.query_data import get_ohlc_data

# from constant import init_cerebro_object
from base_strategy import *

import pandas as pd
import influxdb_client
from influxdb_client import QueryApi
# from database_utils.query_data import get_ohlc_data, get_order_flow
import backtrader.strategies as btstrats

token = "mytoken"
org = "myorg"
url = "http://localhost:8088"
bucket = "VPS-Stock"


GROQ_API_KEY= ""
client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
read_api = client.query_api()
client = Groq(api_key=GROQ_API_KEY)

In [3]:
def get_code_from_text(text):
    code_segement = text.split("```")[1]
    if "```python" in text:
        code_segement = code_segement[6:]
    return code_segement.strip()

In [4]:
# Stock in World
import yfinance as yf
data = yf.download("AAPL", period = "3y")
data["Date"] = data.index
data["Date"] = data["Date"].apply(
        lambda dt: dt.replace(tzinfo=None)
            )
data.head(5)
data = data.resample("5min", on="Date", label="right").agg(
            {
                "Open": "first",
                "High": "max",
                "Low": "min",
                "Close": "last",
                "Volume": "sum",
            }
        ).reset_index()
data = data.dropna()
list_of_data = [bt.feeds.PandasData(
                dataname=data, datetime="Date", timeframe=bt.TimeFrame.Minutes
                )]
print('backtest DF')
data.head(5)

[*********************100%%**********************]  1 of 1 completed

backtest DF


,Date,Open,High,Low,Close,Volume
0,2021-05-24 00:05:00,126.010002,127.940002,125.940002,127.099998,63092900
288,2021-05-25 00:05:00,127.820000,128.320007,126.320000,126.900002,72009500
576,2021-05-26 00:05:00,126.959999,127.389999,126.419998,126.849998,56575900
864,2021-05-27 00:05:00,126.440002,127.639999,125.080002,125.279999,94625600
1152,2021-05-28 00:05:00,125.570000,125.800003,124.550003,124.610001,71311100


In [5]:
with open("base_strategy.py", "r") as f:
    base_strats = "\n".join(f.readlines())
base_strats

'import backtrader as bt\n\n\n\n\n\nclass BaseStrategy(bt.Strategy):\n\n    def __init__(self, debug=True):\n\n        """\n\n        final signal: 1 -> long, 0 -> neutral ->, -1 sell\n\n        """\n\n        # To keep track of pending orders and buy price/commission\n\n        self.countBuy = 0\n\n        self.countSell = 0\n\n        self.final_signal = None\n\n        self.debug = debug\n\n\n\n    def log(self, txt, dt=None):\n\n        """Logging function fot this strategy"""\n\n        if self.debug:\n\n            dt_day = self.datas[0].datetime.date(0)\n\n            dt_value = dt or self.datas[0].datetime.time(0)\n\n            print("%sT%s, %s" % (dt_day, dt_value.isoformat(), txt))\n\n        else:\n\n            pass\n\n\n\n    def notify_order(self, order):\n\n        if order.status in [order.Submitted, order.Accepted]:\n\n            # Buy/Sell order submitted/accepted to/by broker - Nothing to do\n\n            return\n\n        if order.status in [order.Completed]:\n\n

# My code

In [6]:
from typing import Union

def init_cerebro_object(strategy, list_of_data, strategy_params = None, verbose = False, stake = Union[int, None], cash = Union[int, None]):
    cerebro = bt.Cerebro()

    # Adding strategy
    cerebro.addstrategy(strategy = strategy)

    # Adding data
    for data in list_of_data:
        cerebro.adddata(data)

    # Adding initial cash
    cerebro.broker.set_cash(cash)

    return cerebro



list_of_data = [bt.feeds.PandasData(
                dataname=data, datetime="Date", timeframe=bt.TimeFrame.Minutes
                )]

def run_cerebro(strategy, list_data):
    cerebro = init_cerebro_object(strategy=strategy, list_of_data=list_data, 
                                    strategy_params=None, verbose=False, 
                                    stake=100, cash=20_000)
    thestrats = cerebro.run()
    return thestrats

In [7]:
import openai
import re


def chat_query(*args):

    client = openai.OpenAI(
            base_url = "https://api.endpoints.anyscale.com/v1",
            api_key="esecret_mxhfybv3ehvz954hihatkyqh24"
            )

    chat_completion = client.chat.completions.create(
        model = "meta-llama/Meta-Llama-3-70B-Instruct",
        messages = [
            {
                "role": "user", "content": args[0]
            }
        ],
        temperature = 0.1
    )
    return chat_completion.choices[0].message.content

def chat_revise(code, revise = Union[str, None]):
    if revise:
        revise_system = f"""
        You are a python developer that intent to make a workable strategy from human requirement.
        Your task is to create a new BackTestStrategy that follow below msg
        Note: Only return class strategy with your change in execute function

        Please revise the below script based on requests: {revise}
        {code}    
        """
        return chat_query(revise_system)
    else:
        return code


def extract_error_message(error):
    error_lines = str(error).split('\n')
    for line in error_lines:
        if 'Error' in line or 'Exception' in line:
            return line.strip()
    return error_lines[-1].strip()

def check_hold_counter(code):
    code_base = str(code).split('\n')
    for c in code_base:
        if 'hold_counter' not in c or 'position' not in c:
            return True
    
    return False



def auto_debug_code(code,data, max_attempts=5):
    attempt = 0
    while attempt < max_attempts:
        try:

            if check_hold_counter(code):
                revise = "I don't see where you close the position after hold_counter equal 0"
                code = chat_revise(code, revise)
            
            exec(get_code_from_text(code), globals())
            thestrats = run_cerebro(strategy = BackTestStrategy, list_data = data)
            print("Code executed successfully!")
            return code

        except Exception as e:
            error_message = extract_error_message(e)
            print(f"Attempt {attempt+1} failed with error: {error_message}")
            
            debug_prompt = f"""
            You are a Python developer and try to fixing the error when the code given. 
            The following code has an error:\n\n{code}\n\nError: {error_message}\n\nPlease fix the error.
            
            Just fix the code following instruction:
                And below is the structure of the codebase, you can read the MovingAverageStrategy as an example to make your strategy but only change the `execute` function.
                Please name your class strategy as default name is BackTestStrategy
                ----------------------------
                "{base_strats}"
            """
            # code = generate_code(debug_prompt)

            code = chat_query(debug_prompt)
            print(f"Generated new code:\n{code}\n")
            
            attempt += 1
            
    print("Maximum attempts reached. Code could not be debugged.")
    return None


def python_code_generate(*args):
    system = args[0]
    data = args[1]
    try_script = chat_query(system)
    try:
        fixed_script = auto_debug_code(try_script, data, max_attempts=5)
        if fixed_script is None:
            raise Exception('Code could not be fixed!!!')
    except Exception as e:
        print("Return the first generate code.")
        fixed_script = try_script
    
    return fixed_script




In [8]:
prompt = "Please create a strategy that following buy and sell signal of Golden Cross strategy."
prompt = "Please create a strategy that buy position when MA1 > MA50, and close position after 10 cycle."
prompt = "Please create a strategy that buy when RSI is below 50 and sell when RSI is above 80."
prompt = "Please create a strategy that buy when above upperband and below lowerband of Bollinger Bands"

system = f"""
You are a python developer that intent to make a workable strategy from human requirement.
Your task is to create a new BackTestStrategy that follow below msg
Note: Only return class strategy and change `execute` function only, nothing else


{prompt}
And below is the structure of the codebase, you can read the MovingAverageStrategy as an example to make your strategy but only change the `execute` function.
Please name your class strategy as default name is BackTestStrategy
----------------------------
"{base_strats}"
"""

code_script = python_code_generate(system, list_of_data)
print(code_script)

Code executed successfully!
Here is the revised `BackTestStrategy` class that closes the position when `hold_counter` equals 0:
```
class BackTestStrategy(BaseStrategy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bb = bt.ind.BollingerBands(self.data.close, period=20, devfactor=2)
        self.hold_counter = 0

    def execute(self):
        if self.data.close > self.bb.bbh:
            if self.hold_counter == 0:
                self.buy()
                self.hold_counter = 5  # hold for 5 bars
            return 1
        elif self.data.close < self.bb.bbl:
            if self.hold_counter == 0:
                self.sell()
                self.hold_counter = 5  # hold for 5 bars
            return -1
        else:
            self.hold_counter -= 1
            if self.hold_counter == 0:
                self.close()  # close the position
            return 0
```
Here's what's changed:

* I added a `hold_counter` attribute to the stra

# End my code

In [9]:
# exec(get_code_from_text(final_script))
exec(get_code_from_text(code_script))

In [10]:
list_of_data = [bt.feeds.PandasData(
                dataname=data, datetime="Date", timeframe=bt.TimeFrame.Minutes
                )]
cerebro = init_cerebro_object(strategy=BackTestStrategy, list_of_data=list_of_data, strategy_params=None, verbose=False, stake=100, cash=20_000)
thestrats = cerebro.run()
thestrat = thestrats[0]

In [11]:
# Print out the final result
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
print(
    "Total point return: ",
    (cerebro.broker.getvalue() - cerebro.broker.startingcash),
)

Final Portfolio Value: 20000.00
Total point return:  0.0


In [12]:
figs = cerebro.plot(
        iplot=False, 
        style="pincandle", width=60 * 10, height=40 * 10, 
        # dpi=1000
    )

In [13]:
for each in thestrat.analyzers:
    each.print()

In [14]:
figs = cerebro.plot(
        iplot=False, style="pincandle", width=32 * 10, height=18 * 10, dpi=300
    )

: 